In [1]:
PATH = '/Users/admin/Downloads/riiid-test-answer-prediction/'
import pandas as pd
import datatable as dt
import random

In [1]:
train = dt.fread('./input/train.csv'
                            ,skip_to_line = 60 * 1000000
                            ,max_nrows = 40 * 1000000
                            ).to_pandas()

train.columns = ['row_id', 'timestamp', 'user_id', 'content_id', 'content_type_id',
               'task_container_id', 'user_answer', 'answered_correctly',
               'prior_question_elapsed_time', 'prior_question_had_explanation']

    
train = train.loc[train.content_type_id == 0]
train = train[['user_id', 'content_id', 'timestamp', 'answered_correctly']]

In [2]:
tmp_ids = train.groupby('content_id').size().sort_values(ascending=False)[:7000].index
a_cols = list(set(list(tmp_ids[:500]) + random.sample(list(tmp_ids[500:]), 500)))
count = 0
final_output = None
for a in a_cols:
    b_cols = list(set(random.sample(list(tmp_ids[:100]), 10) + random.sample(list(tmp_ids[100:]), 10)))
    for b in b_cols:
        
        # Don't do for same content_id
        if a == b:
            continue

        # Get rows that have the questions
        tmp = train.loc[train['content_id'].isin([a, b])]

        # Keep only users who saw both questions one time each
        tmp_gb = tmp.groupby('user_id')['content_id'].agg(n_unique = 'nunique', count = 'count')
        tmp_gb = tmp_gb.loc[(tmp_gb['n_unique'] == 2) & (tmp_gb['count'] == 2)]
        tmp = tmp.loc[tmp['user_id'].isin(tmp_gb.index)]
        
        if tmp.shape[0] < 200:
            continue

        # Keep only the second occurence
        tmp['prev_timestamp'] = tmp.groupby('user_id')['timestamp'].shift()
        tmp['prev_ac'] = tmp.groupby('user_id')['answered_correctly'].shift()
        tmp = tmp.loc[~tmp['prev_ac'].isna()]


        # Attach to pandas
        tmp_output = pd.DataFrame({
            'a' : [a]
            ,'b' : [b]
            ,'a_right_b_mean' : [tmp.loc[(tmp['content_id'] == b) 
                                        & (tmp['prev_ac'] == 1)]['answered_correctly'].mean()]
            ,'a_right_b_size' : [len(tmp.loc[(tmp['content_id'] == b) 
                                     & (tmp['prev_ac'] == 1)]['answered_correctly'])]
            ,'a_wrong_b_mean' : [tmp.loc[(tmp['content_id'] == b) 
                                        & (tmp['prev_ac'] == 0)]['answered_correctly'].mean()]
            ,'a_wrong_b_size' : [len(tmp.loc[(tmp['content_id'] == b) 
                                            & (tmp['prev_ac'] == 0)]['answered_correctly'])]
            ,'b_right_a_mean' : [tmp.loc[(tmp['content_id'] == a) 
                                      & (tmp['prev_ac'] == 1)]['answered_correctly'].mean()]
            ,'b_right_a_size' : [len(tmp.loc[(tmp['content_id'] == a) 
                                            & (tmp['prev_ac'] == 1)]['answered_correctly'])]
            ,'b_wrong_a_mean' : [tmp.loc[(tmp['content_id'] == a) 
                                        & (tmp['prev_ac'] == 0)]['answered_correctly'].mean()]
            ,'b_wrong_a_mean' : [tmp.loc[(tmp['content_id'] == a) 
                                        & (tmp['prev_ac'] == 0)]['answered_correctly'].mean()]
            ,'b_wrong_a_size' : [len(tmp.loc[(tmp['content_id'] == a) 
                                            & (tmp['prev_ac'] == 0)]['answered_correctly'])]          
        })
        if final_output is None:
            final_output = tmp_output
        else:
            final_output = final_output.append(tmp_output)

        count += 1
        if count % 1000 == 0:
            print(count)



1000
2000
3000
4000
5000
6000
7000
8000
skip
skip
skip
skip
9000
skip
10000
skip
11000
12000
13000
14000
skip
15000
16000
17000
18000
skip
19000


In [3]:
content_id_means = train.groupby('content_id')[['answered_correctly']].mean()\
                        .rename(columns={'answered_correctly' : 'mean'})
d = final_output.merge(content_id_means.rename(columns={'mean' : 'mean_a'})
                       ,left_on='a', right_on='content_id', how='left')\
                .merge(content_id_means.rename(columns={'mean' : 'mean_b'})
                       ,left_on='b', right_on='content_id', how='left')
d.to_csv('./output/cond_probs_' + str(d.shape[0]) + '.csv', index=False)

Explore

In [2]:
# Get output csv
d = pd.read_csv('./output/cond_probs_19982.csv')
cols = ['a', 'b', 'a_right_b_mean', 'a_right_b_size'
        ,'a_wrong_b_mean', 'a_wrong_b_size', 'mean_b']
d = d[cols]
d['b_dif'] = (d['a_right_b_mean'] - d['a_wrong_b_mean']).abs()
d = d.loc[(d['a_right_b_size'] > 150) & (d['a_wrong_b_size'] > 150)]
d = d.loc[d.b_dif > .1]
d.sort_values(['b_dif'], ascending=False, inplace=True)
d = d.groupby('b').head(1)
d = d.groupby('a').head(1)
d.reset_index(inplace=True, drop=True)

# Transform into a dictionary
cond_probs = {
    'content_id_prev' : list(d['a'])
    ,'content_id_keys' : {}
    ,'prev_right_current_mean' : list(round(d['a_right_b_mean'], 4))
    ,'prev_wrong_current_mean' : list(round(d['a_wrong_b_mean'], 4))
}
for i in range(d.shape[0]):
    cond_probs['content_id_keys'][d.loc[i, 'b']] = i
d

,a,b,a_right_b_mean,a_right_b_size,a_wrong_b_mean,a_wrong_b_size,mean_b,b_dif
0,6116,4108,0.825739,1997,0.447479,5236,0.511606,0.378260
1,4691,6119,0.642534,221,0.318182,220,0.445286,0.324352
2,5588,5616,0.689708,651,0.389595,865,0.419784,0.300113
3,593,853,0.692868,687,0.407285,604,0.409489,0.285583
4,412,738,0.444976,1254,0.166667,252,0.372211,0.278309
...,...,...,...,...,...,...,...,...
353,4533,3249,0.771300,446,0.670391,358,0.661566,0.100909
354,1265,5352,0.352584,658,0.251870,401,0.309241,0.100713
355,5154,613,0.943396,265,0.843023,172,0.895092,0.100373
356,6266,730,0.772436,312,0.672199,241,0.641749,0.100237


In [3]:
# Pickle it
import pickle
with open('./output/test_cond_probs.pickle', 'wb') as handle:
    pickle.dump(cond_probs, handle, protocol=pickle.HIGHEST_PROTOCOL)